<h1>Import Models</h1>

In [1]:
import pandas as pd 
import os
import shutil
import torch
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
import PIL
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import sklearn
from sklearn import metrics
from tqdm import tqdm

<h1>Pandas</h1>

In [2]:
!nvidia-smi

Tue Jul 30 12:35:37 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           On  |   00000000:02:00.0 Off |                    0 |
| N/A   32C    P0             25W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
os.getcwd()

'/ibex/user/abduam0d/3D-Printer-Error-Detection'

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
dataset_path = 'Auto_ZMJ2N'
dataset_os = os.path.join(dataset_path, 'classes.csv')
img_folder_train = os.path.join(dataset_path, 'train/')
img_folder_test = os.path.join(dataset_path, 'test/')
print(dataset_os)


Auto_ZMJ2N/classes.csv


<h1>Create Custom Dataset</h1>

In [6]:



class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, img_folder, transform) -> None:
        super().__init__()
        self.dataset = pd.read_csv(dataset)
        self.img_folder = img_folder
        self.transform = transform
    def __len__(self) -> int:
        return len(self.dataset)
    def __getitem__(self, index):
        img_path = f"{self.img_folder + self.dataset.loc[index]['filename']}"
        label = self.dataset.loc[index][1:]
        img = PIL.Image.open(img_path)
        img = self.transform(img)
        return img, torch.tensor(label).float()

<h1>Building Model and Dataset</h1>

In [7]:
model = models.resnet50(pretrained=True)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset = CustomDataset(dataset_os, img_folder_train, transform= transform)

test_dataset = datasets.ImageFolder(img_folder_test,transform=transform)



/ibex/user/abduam0d/conda-environments/temp_env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/ibex/user/abduam0d/conda-environments/temp_env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<h1>Dataloader</h1>

In [8]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)


<h1>Modifying Model</h1>

In [9]:
params = model.parameters()
for param in params:
    param.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(2048, 512),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(512, 4),
)


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [11]:
model.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [12]:

for epoch in tqdm(range(18), desc="Training", unit="epoch"):
    running_loss = 0.0
    total = 0
    correct = 0
    train_loss_history = []
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        labels = torch.argmax(labels, dim=1)  
        correct += (predicted == labels).sum().item()
        accuracy = correct / total * 100
        
    print(f'epoch: {epoch + 1} loss: {running_loss / len(train_loader):.3f}, accuracy: {accuracy:.2f}%')
    running_loss = 0.0
    total = 0
    correct = 0

print('Finished Training')

Training:   0%|                                                                                               | 0/18 [00:00<?, ?epoch/s]/tmp/ipykernel_2991767/2836196652.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return img, torch.tensor(label).float()
/ibex/user/abduam0d/conda-environments/temp_env/lib/python3.12/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv2d(input, weight, bias, self.stride,
Training:   6%|████▊                                                                                  | 1/18 [00:10<02:55, 10.33s/epoch]

epoch: 1 loss: 1.288, accuracy: 42.23%


Training:  11%|█████████▋                                                                             | 2/18 [00:20<02:42, 10.14s/epoch]

epoch: 2 loss: 1.055, accuracy: 60.59%


Training:  17%|██████████████▌                                                                        | 3/18 [00:30<02:31, 10.08s/epoch]

...

epoch: 16 loss: 0.388, accuracy: 87.03%


Training:  94%|█████████████████████████████████████████████████████████████████████████████████▏    | 17/18 [02:48<00:09,  9.74s/epoch]

epoch: 17 loss: 0.361, accuracy: 86.53%


Training: 100%|██████████████████████████████████████████████████████████████████████████████████████| 18/18 [02:58<00:00,  9.92s/epoch]

epoch: 18 loss: 0.332, accuracy: 87.38%
Finished Training


In [19]:
model.eval()

correct = 0
total = 0
all_labels = []
all_preds = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        modified_tensor = labels.clone()
        modified_tensor[labels == 1] = 3
        modified_tensor[labels == 2] = 1
        modified_tensor[labels == 3] = 2
        real = modified_tensor
        total += labels.size(0)
        all_labels.extend(real.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())
        correct += (predicted == real).sum().item()

accuracy = correct / total * 100
print(f'Test Accuracy: {accuracy:.2f}%')
print(len(all_labels), len(all_preds))
        

Test Accuracy: 94.66%
356 356
